# Increasing amount Investment(증액투자법)

|레벨|스타일|기대CAGR|매수전략|매도전략|
|:--:|:--:|:--:|:--:|:--:|
|초급|자산배분|코스피 + 1~2%(20년 투자할 경우)|- 전체 자산을 3년채 국채에 투자<br/><br/>- 국채에서 나오는 투자 수익을 코스피 지수에 투자|없음|

#Import module

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

# Get Daily Prices

##KODEX_200 (daily)

KODEX 200 일별 데이터를 네이버 금융에서 크롤링 해온 후, `kodex_200` 변수에 담습니다.

In [ ]:
kodex_200 = pd.DataFrame()

kodex_url = 'https://finance.naver.com/item/sise_day.nhn?code=069500' 

for page in tqdm(range(1, 100)) : 
    pg_url = '{url}&page={page}'.format(url=kodex_url, page=page)

    pg_url = requests.get(pg_url, headers={'User-agent' : 'Mozilla/5.0'}).text
    kodex_200 = kodex_200.append(pd.read_html(pg_url)[0], ignore_index=True)



In [ ]:
kodex_200 = kodex_200.dropna()
kodex_200.columns = ['date', 'close', 'over_the_day', 'open', 'high', 'low', 'volume']
kodex_200.date = pd.to_datetime(kodex_200.date)

print('출처: ', kodex_url)
print(kodex_200.shape)
kodex_200.head()

In [ ]:
type(kodex_200.iloc[0]['date'])

## GOV_3Y(daily)

국채3년물 상품을 네이버 금융에서 크롤링합니다. 크롬의 검사(Inspection) 기능을 사용해 개발자 도구를 열고 크롤링해오려는 데이터 표의 source URL을 먼저 변수로 지정합니다. 크롤링해온 일별 데이터를 `gov_3` 에 DataFrame으로 담습니다.

In [ ]:
gov_3 = pd.DataFrame()
gov_url = 'https://finance.naver.com//item/sise_day.nhn?code=114260'
for page in tqdm(range(1, 100)):
    pg_url = '{url}&page={page}'.format(url=gov_url, page=page)
    pg_url = requests.get(pg_url, headers={'User-agent' : 'Mozilla/5.0'}).text
    gov_3 = gov_3.append(pd.read_html(pg_url)[0], ignore_index=True)

In [ ]:
gov_3 = gov_3.dropna()
gov_3.columns = ['date', 'close', 'over_the_day', 'open', 'high', 'low', 'volume']
gov_3.date = pd.to_datetime(gov_3.date)
gov_3

#Turn to Monthly Data

##KODEX_200 & GOV_3Y (monthly)

2017년 ~ 2021년까지, 1월 ~ 12월까지 `for 반복문`을 사용해 해당 연|월의 말일 데이터만 필터링해 각각 `kodex_monthly`, `gov_monthly`에 담습니다.

In [ ]:
kodex_monthly = pd.DataFrame()
gov_monthly = pd.DataFrame()
for y in tqdm(range(2017, 2022)):
    for m in range(1, 13) : 
        kodex_monthly = kodex_monthly.append(kodex_200[(kodex_200['date'].dt.year == y) & (kodex_200['date'].dt.month == m)].head(1))
        gov_monthly =  gov_monthly.append(gov_3[(gov_3.date.dt.year == y) & (gov_3.date.dt.month == m)].head(1))


range(0,len(gov_monthly))

# list comprehension

kodex_monthly.index = [x for x in range(len(gov_monthly))]
gov_monthly.index = [x for x in range(len(gov_monthly))]

# Backtesting

In [ ]:
# psuedo function
def backtest(beginning_asset = 100000): # 최초 금액은 $100,000로 시작합니다
    
    asset = [beginning_asset] # 최종적으로 반환할 월별 자산 데이터가 담길 리스트를 선언합니다
    kodex_shares = 0 # 첫 시점에서 KODEX 주식 수는 0주입니다.
    
    for i in range(len(gov_monthly)-1): # 월별 데이터의 개수만큼 반복합니다
        gov_shares = beginning_asset / gov_monthly.iloc[i].close # i번째 회차에 투자해야 할 국채 3년물 주식 수를 구합니다. 
        gov_ending_value = gov_shares * gov_monthly.iloc[i+1].close # 위에서 구한 주식 수에 i+1번째 회차의 국채 3년물 주가를 곱해 자산가치를 구합니다.

        # 국채 3년물 투자금의 기초자산 대비 추가 수익률을 구합니다.
        surplus = gov_ending_value - 100000
        if surplus > 0 : # 수익이 발생
            kodex_shares += surplus / kodex_monthly.iloc[i+1].close # 추가적으로 발생한 수익금으로 매수가능한 KODEX 주식 수를 계산합니다.
        
        # 해당 시점 초기자금과 매수한 주식금액을 합산해 최종 자산을 구합니다.
        total_ending_value = beginning_asset + (kodex_shares * kodex_monthly.iloc[i+1].close)
        asset.append(total_ending_value) # 총 자산가치를 asset 리스트에 담습니다.
    return asset

In [ ]:
result = backtest()
result

In [ ]:
back_test = {
    'date' : gov_monthly.date, 
    'backtest' : result,
    'KODEX_200' : kodex_monthly.close / kodex_monthly.iloc[0].close * 100000,
    'GOV_3Y' : gov_monthly.close / gov_monthly.iloc[0].close * 100000
} 

In [ ]:
back_test = pd.DataFrame(back_test)
back_test

#Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
plt.figure(figsize=(18, 4))
plt.plot(back_test.date, back_test.backtest, label='Backtesting')
plt.plot(back_test.date, back_test.KODEX_200, label='KODEX_200')
plt.plot(back_test.date, back_test.GOV_3Y, label='GOV_3Y')

plt.legend(loc='best')
plt.grid()
plt.title('Backtesting Portfolio & Benchmarks', fontsize=25)
plt.xlabel('Year', fontsize=18)
plt.ylabel('Asset Amount', fontsize=18 )


plt.show()

#Compute CAGR

각 자산별 Compounded Annual Growth Rate를 계산해보겠습니다,

##backtest

In [ ]:
start, end = result[0], result[-1]
cagr = ((end/start)**(1/len(set(kodex_monthly.date.dt.year)))-1)*100
print('%.2f'%cagr,'%')

##kodex 200

In [ ]:
start, end = kodex_monthly.iloc[0].close, kodex_monthly.iloc[-1].close
cagr = ((end/start)**(1/len(set(kodex_monthly.date.dt.year)))-1)*100
print('%.2f'%cagr,'%')

##gov 3Y

In [ ]:
start, end = gov_monthly.iloc[0].close, gov_monthly.iloc[-1].close
cagr = ((end/start)**(1/len(set(kodex_monthly.date.dt.year)))-1)*100
print('%.2f'%cagr,'%')

# Compute Statistics

`numpy`모듈을 사용해 통계치를 계산합니다

In [ ]:
import numpy as np


In [ ]:
# numpy -> 모집단 표준편차(N)
#np.std(back_test)

In [ ]:
# pandas -> 표본 표준편차(n-1)
back_test.std()

In [ ]:
back_test.mean()

In [ ]:
back_test.std() / back_test.mean()

In [ ]:
back_test.backtest.head()

`diff`를 사용해 매월 자산액의 차이를 계산한 후 전월대비 수익률을 계산합니다.

In [ ]:
# 월 별 수익률
back_test.backtest.diff() # diff() 차이를 계산


In [ ]:

back_test['RoR'] = back_test.backtest.diff() / back_test.backtest * 100
back_test['KODEX_RoR'] = back_test.KODEX_200.diff() / back_test.KODEX_200 * 100
back_test['GOV_RoR'] = back_test.GOV_3Y.diff() / back_test.GOV_3Y * 100

In [ ]:
back_test = back_test.fillna(0)
#back_test

수익률의 표준 편차를 계산합니다.


In [ ]:
back_test.std() / back_test.mean() * 100

수익률의 표준편차가 최대치가 되는 시점을 찾습니다.


In [ ]:
back_test[back_test.RoR == max(back_test.RoR)].date.values[0]

수익률의 표준편차가 최소치가 되는 시점을 찾습니다.


In [ ]:
back_test[back_test.RoR == min(back_test.RoR)].date.values[0]

백테스트 수익률 분포를 히스토그램으로 나타냅니다.

In [ ]:
plt.figure()

plt.hist(back_test.RoR, bins=100)

plt.show()

KODEX 수익률 분포를 히스토그램으로 나타냅니다.

In [ ]:
plt.figure()

plt.hist(back_test.KODEX_RoR, bins=100)

plt.show()

국채3년물 수익률 분포를 히스토그램으로 나타냅니다.

In [ ]:
plt.figure()

plt.hist(back_test.GOV_RoR, bins=100)

plt.show()